In [ ]:
Metrics of Scoring
F1 score
Cohen’s kappa



In [2]:
#load libraries
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.metrics import roc_curve, auc, recall_score, precision_score, f1_score, confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [1]:
import  lightgbm as lgbm 
import xgboost as xgb 
import catboost as cat 

In [4]:
# Load data
df = pd.read_csv('challenge_1_gut_microbiome_data.csv')

## Data Overview

In [5]:
df.head()

,Unnamed: 0,Bacteria-1,Bacteria-2,Bacteria-3,Bacteria-4,Bacteria-5,Bacteria-6,Bacteria-7,Bacteria-8,Bacteria-9,...,Bacteria-1086,Bacteria-1087,Bacteria-1088,Bacteria-1089,Bacteria-1090,Bacteria-1091,Bacteria-1092,Bacteria-1093,Bacteria-1094,disease
0,Sample-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Disease-2
1,Sample-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,17111.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Disease-2
2,Sample-3,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,214.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Disease-2
3,Sample-4,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Disease-2
4,Sample-5,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1932.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,Disease-2


In [7]:
#drop first column
df.drop('Unnamed:', axis=1, inplace=True)

KeyError: "['Unnamed:'] not found in axis"